In [1]:
import pickle
import sqlite3
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

# Load

In [2]:
conn = sqlite3.connect('../loteca/data/raw/betexplorer/db.sqlite3')
betexp = pd.read_sql_query('SELECT id, date, teamH, teamA, score, scoremod FROM matches', conn)
conn.close()

betexp.date = pd.to_datetime(betexp.date, dayfirst=True)
betexp.score = betexp.score.str.strip()
betexp['goalsH'] = [int(score.split(':')[0]) if score else np.nan for score in betexp.score]
betexp['goalsA'] = [int(score.split(':')[1]) if score else np.nan for score in betexp.score]

betexp.head()

,id,date,teamH,teamA,score,scoremod,goalsH,goalsA
0,GCjoqXpn,2018-01-22,Israel W,Finland W,0:0,,0.0,0.0
1,xMispDVu,2017-11-28,Spain W,Austria W,4:0,,4.0,0.0
2,xnyYIzXt,2017-11-28,Portugal W,Italy W,0:1,,0.0,1.0
3,pjD4z32E,2017-11-28,Romania W,Moldova W,3:1,,3.0,1.0
4,nZiE6CQJ,2017-11-28,Netherlands W,Ireland W,0:0,,0.0,0.0


In [3]:
loteca = pd.read_pickle('../data/pre/lotecas_matches.pkl')
loteca = loteca[loteca.roundno >= 366]  # exclude old rounds
loteca.head()

,roundno,gameno,date,teamH,goalsH,teamA,goalsA,happened
5110,366,1,2009-06-07,CRUZEIRO/MG,1,INTERNACIONAL/RS,1,True
5111,366,2,2009-06-07,ATLÉTICO/PR,0,ATLÉTICO/MG,4,True
5112,366,3,2009-06-07,AVAÍ/SC,0,SÃO PAULO/SP,0,True
5113,366,4,2009-06-06,AMÉRICA/RN,2,BRASILIENSE/DF,1,True
5114,366,5,2009-06-06,PONTE PRETA/SP,5,PORTUGUESA DESPORTOS/SP,2,True


In [4]:
with open('../data/interim/teams_ltb.pkl', mode='rb') as fp:
    teamsd = pickle.load(fp)
    
list(teamsd.items())[:5]

[('A B C/RN', {'ABC'}),
 ('ABAETÉ/PA', set()),
 ('ABC/RN', {'ABC'}),
 ('ALBÂNIA', {'Albania'}),
 ('ALECRIM SUB 20/RN', {'Alecrim U20'})]

# Algorithm

In [5]:
loteca_teams = set(loteca.teamH) | set(loteca.teamA)
betexp_teams = set()
for team in loteca_teams:
    if teamsd[team]:
        betexp_teams.add(next(iter(teamsd[team])))
        
betexp_teams = sorted(betexp_teams)
betexp_teams[:10]

['ABC',
 'AS Roma',
 'ASA',
 'Albania',
 'Alecrim',
 'Alecrim U20',
 'Algeria',
 'America MG',
 'America MG U20',
 'America PE']

In [6]:
dates = set()
for date in loteca.date:
    dates.add(date)

dates = sorted(dates)
dates[:10]

[NaT,
 Timestamp('2009-06-06 00:00:00'),
 Timestamp('2009-06-07 00:00:00'),
 Timestamp('2009-06-13 00:00:00'),
 Timestamp('2009-06-14 00:00:00'),
 Timestamp('2009-06-20 00:00:00'),
 Timestamp('2009-06-21 00:00:00'),
 Timestamp('2009-06-27 00:00:00'),
 Timestamp('2009-06-28 00:00:00'),
 Timestamp('2009-07-04 00:00:00')]

In [7]:
betexp.shape

(148101, 8)

In [8]:
betexp_min = betexp[(betexp.teamH.isin(betexp_teams)) | (betexp.teamA.isin(betexp_teams))]
betexp_min = betexp_min[betexp_min.date.isin(dates)]
betexp_min.shape

(16379, 8)

In [9]:
def find_betexplorer_equivalent(loteca_match):    
    row = loteca_match     
    
    # date
    choices = betexp_min    
    choices = choices[choices.date == row.date]
    
    # score
    choices = choices[choices.goalsH == row.goalsH]
    choices = choices[choices.goalsA == row.goalsA]
    
    # teams
    teamsH = teamsd[row.teamH]
    teamsA = teamsd[row.teamA]    
    if teamsH:
        choices = choices[choices.teamH.isin(teamsH)]
    if teamsA:
        choices = choices[choices.teamA.isin(teamsA)]
        
    # return
    if teamsH or teamsA:
        if choices.shape[0] == 0:
            return None
        if choices.shape[0] == 1:
            if add_team(row.teamH, choices.iloc[0].teamH) or add_team(row.teamA, choices.iloc[0].teamA):
                display(row)
                display(choices)
#             add_team(row.teamH, choices.iloc[0].teamH)
#             add_team(row.teamA, choices.iloc[0].teamA)
            return choices.index[0]
        else:
            raise ValueError("Found %s BetExplorer matches for Loteca #%s" % \
                             (choices.shape[0], row.name))
            
def add_team(loteca_string, betexp_string):
    if not teamsd[loteca_string]:
        print("{: >30} > {}".format(loteca_string, betexp_string))
        teamsd[loteca_string].add(betexp_string)
        return True
    else:
        return False
    

dict = OrderedDict()
            
first = True
newteamcnt = False
roundno = 1
while first or newteamcnt:    
    print("Round number %s" % roundno)
    teamcnt = len([1 for v in teamsd.values() if v])
    
    for id, row in loteca.iterrows():
        if id in dict.keys(): continue
        if not row.happened: continue  # deal with these cases later
        betexp_id = find_betexplorer_equivalent(row)
        if betexp_id:
            dict[id] = betexp_id
            
    first = False
    newteamcnt = len([1 for v in teamsd.values() if v]) - teamcnt
    roundno += 1
    
    print("New teams found: %s" % newteamcnt)
    print("Matches found till now: %s" % len(dict))
    
all([len(v) in (0, 1) for v in teamsd.values()])  # check teamsd dictionary integrity

Round number 1
                      SPORT/PE > Sport Recife


roundno                     366
gameno                       13
date        2009-06-07 00:00:00
teamH                  SPORT/PE
goalsH                        4
teamA               FLAMENGO/RJ
goalsA                        2
happened                   True
Name: 5122, dtype: object

,id,date,teamH,teamA,score,scoremod,goalsH,goalsA
22747,Io8dfmf7,2009-06-07,Sport Recife,Flamengo RJ,4:2,,4.0,2.0


              VASCO DA GAMA/RJ > Vasco


roundno                     367
gameno                        4
date        2009-06-13 00:00:00
teamH                GUARANI/SP
goalsH                        0
teamA          VASCO DA GAMA/RJ
goalsA                        0
happened                   True
Name: 5127, dtype: object

...

                  VILA NOVA/MG > Villa Nova MG


roundno                     787
gameno                        7
date        2018-02-17 00:00:00
teamH               CRUZEIRO/MG
goalsH                        1
teamA              VILA NOVA/MG
goalsA                        0
happened                   True
Name: 11010, dtype: object

,id,date,teamH,teamA,score,scoremod,goalsH,goalsA
84076,bwTio95S,2018-02-17,Cruzeiro,Villa Nova MG,1:0,,1.0,0.0


New teams found: 43
Matches found till now: 4227
Round number 2
New teams found: 0
Matches found till now: 4227


True

In [19]:
# http://www.betexplorer.com/soccer/brazil/serie-b-2009/america-rn-betim/zuc6o4ea/
# it's Ipatinga, not Betim
betexp.loc[22800]

id                     zuc6o4ea
date        2009-11-21 00:00:00
teamH                America RN
teamA                     Betim
score                       1:0
scoremod                       
goalsH                        1
goalsA                        0
Name: 22800, dtype: object

In [20]:
# http://www.betexplorer.com/soccer/brazil/campeonato-paulista-2010/santo-andre-barueri/d4IyAGi2/
# It's Grêmio Prudente, not Barueri
betexp.loc[18934]

id                     d4IyAGi2
date        2010-04-18 00:00:00
teamH               Santo Andre
teamA                   Barueri
score                       1:2
scoremod                       
goalsH                        1
goalsA                        2
Name: 18934, dtype: object

In [21]:
# http://www.betexplorer.com/soccer/brazil/serie-b-2011/barueri-portuguesa/4rHNNkEH/
# This one is right, but it contradicts with the first one
betexp.loc[32436]

id                     4rHNNkEH
date        2011-06-11 00:00:00
teamH                   Barueri
teamA                Portuguesa
score                       1:1
scoremod                       
goalsH                        1
goalsA                        1
Name: 32436, dtype: object

In [23]:
# we should have caught Fernadopolis
# OH, it is because of the state
betexp.loc[83272]

id                     OvimZR1o
date        2018-01-06 00:00:00
teamH         Fernandopolis U20
teamA               Guarani U20
score                       2:4
scoremod                       
goalsH                        2
goalsA                        4
Name: 83272, dtype: object

In [10]:
print(len(dict))
print(loteca.shape[0])
print(len(dict) / loteca.shape[0])

# todo: one day before and one day after

4227
5908
0.715470548408937
